# Calculate the technology specific LCA impact 
# end point level

0. import brightway2 results for the coal exit scenarios
0. calcualte the energy market electricity high voltage mix RECIPE total, HH, ED and RD
(indicator x scenarios x period)

0. calculate the mid to endpoint DALYS, and species.year

0. do the same for the other SE carriers 
    1. coarse match of REMIND and brightway technologies on REMIND SE level
    2. calculate the RECIPE total (3) for all these technologies in table form
       (technologies x indicator x scenarios x period)

1. use these tables to do a post processing (R) to aggregate and monetize the DALYS and monetize the ED
     1. read in results of this script
     2. use conversion factors for ecoinvent to REMIND
     3. use a REMIND activity mapping
     4. monetize DALYS and PDFs
    
further developments
1. adjust mid to end pop with SSP pop data country specific?
2. adjust the land occupation indicator with country specific species data
3. disaggregate the land occupating indicator 
4. update to RECIPE 2016 when available
5. dont use the global mean for SE technologies
6. automatic scenario name mapping

the se regional mapping is global

# import the brightway2 results for the coal exit scenarios

In [3]:
import os
import pandas as pd

In [4]:
path="C:\\Users\\rauner\\Documents\\PIK\\holi_coal_exit\\rmnd-lca\\"
os.chdir(path)

In [5]:
from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

In [6]:
import brightway2 as bw
%run notebooks\\initialize_notebook.ipynb
from wurst.ecoinvent import filters
from constructive_geometries import Geomatcher

C:\Users\rauner\AppData\Local\Continuum\miniconda3\lib\site-packages\constructive_geometries\cg.py:77: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


In [7]:
from wurst import searching as ws

In [ ]:
from helpers import eimod, lcahelp, ei2rmnd

In [9]:
bw.restore_project_directory("data\\brightway2-project-coal_exit-backup.08-January-2019-12-24PM.tar.gz")

Restoring project backup archive - this could take a few minutes...


'coal_exit'

In [10]:
bw.projects.set_current("coal_exit")
project = 'coal_exit'

## create a scenarios dictionary

In [11]:
database_dict = {}
database_dict['ecoinvent_added_technologies'] = {'year' : 2015 , 'scenario' : 'ecoinvent'}

for year in [ 2020, 2030,2040, 2050]:
    for scenario in ['Ref','RCP26','INDC','coal_exit']:
        if year == 2015 and scenario != 'ecoinvent_added_technologies':
            continue
        db_name = 'coal_exit_' + scenario + '_' + str(year)
        database_dict[db_name] = {'year' : year , 'scenario' : scenario}

In [12]:
database_dict

{'ecoinvent_added_technologies': {'year': 2015, 'scenario': 'ecoinvent'},
 'coal_exit_Ref_2020': {'year': 2020, 'scenario': 'Ref'},
 'coal_exit_RCP26_2020': {'year': 2020, 'scenario': 'RCP26'},
 'coal_exit_INDC_2020': {'year': 2020, 'scenario': 'INDC'},
 'coal_exit_coal_exit_2020': {'year': 2020, 'scenario': 'coal_exit'},
 'coal_exit_Ref_2030': {'year': 2030, 'scenario': 'Ref'},
 'coal_exit_RCP26_2030': {'year': 2030, 'scenario': 'RCP26'},
 'coal_exit_INDC_2030': {'year': 2030, 'scenario': 'INDC'},
 'coal_exit_coal_exit_2030': {'year': 2030, 'scenario': 'coal_exit'},
 'coal_exit_Ref_2040': {'year': 2040, 'scenario': 'Ref'},
 'coal_exit_RCP26_2040': {'year': 2040, 'scenario': 'RCP26'},
 'coal_exit_INDC_2040': {'year': 2040, 'scenario': 'INDC'},
 'coal_exit_coal_exit_2040': {'year': 2040, 'scenario': 'coal_exit'},
 'coal_exit_Ref_2050': {'year': 2050, 'scenario': 'Ref'},
 'coal_exit_RCP26_2050': {'year': 2050, 'scenario': 'RCP26'},
 'coal_exit_INDC_2050': {'year': 2050, 'scenario': 'INDC

## calcualte the LCA for all scenarios and years for all SE technologies

In [13]:
#select the modeled processes
#this is a crude mapping of of REMIND SE to ecoinvent progcesses
#we need to adjust the units later in the post processing to match EJ

In [14]:
# see data\ecoinventtoREMINDtechnmap.csv for SE technology mapping
# we use the electricity generation processes as a base and check what the input is to adjust them accordingly
# see the A matrix of the elec processes for the input A.technology.final.technology.list.variable.csv
# ergo if a kWh hard coal needs 5kg of coal we:

# elec to raw input = /5
# 1kWh to 1EJ =  /3.6*10^12
# elec hard coal to se hard coal /3.6*10^12 /5kg

# this factor is in the conv column of the ecoinventtoREMINDtechmap.csv

In [15]:
# se _filter filters for electricity high voltage and all the other SE ecoinvent processes mapped above

In [16]:
se_filter = [ws.either(*[                                                  
                                                                           ws.equals('name', 'heat and power co-generation, biogas, gas engine'),
                                                                           ws.equals('name', 'electricity production, natural gas, combined cycle power plant'),
                                                                           ws.equals('name', 'electricity production, hard coal'),
                                                                           ws.equals('name', 'electricity production, deep geothermal'),
                                                                           ws.equals('name', 'market for hydrogen, liquid'),
                                                                           ws.equals('name', 'electricity production, oil'),
                                                                           ws.equals('name', 'softwood forestry, mixed species, sustainable forest management')])]

In [17]:
electricity_markets = {}
se_technologies = {}

for db_name in database_dict:
    electricity_markets[db_name] = [x for x in ws.get_many(Database(db_name), *[*eimod.electricity_market_filter_high_voltage, ws.exclude(ws.contains('name', 'Swiss Federal Railways')),
                                                                                ws.exclude(ws.contains('name', 'label-certified'))])]
    se_technologies[db_name] = [x for x in ws.get_many(Database(db_name), *[*se_filter])]

## choose the RECIPE categories, we use 2008 with updated 2016 where direct matches are possible

calculate all the ReCiPe Midpoint (H) 2008 version and multiply with mid to endpoints to get DALYS (HH), Species.year (ET) and maybe resources.

In [18]:
method=[m for m in methods if "'ReCiPe Midpoint (H)'," in str(m)]
method

[('ReCiPe Midpoint (H)', 'agricultural land occupation', 'ALOP'),
 ('ReCiPe Midpoint (H)', 'climate change', 'GWP100'),
 ('ReCiPe Midpoint (H)', 'fossil depletion', 'FDP'),
 ('ReCiPe Midpoint (H)', 'freshwater ecotoxicity', 'FETPinf'),
 ('ReCiPe Midpoint (H)', 'freshwater eutrophication', 'FEP'),
 ('ReCiPe Midpoint (H)', 'human toxicity', 'HTPinf'),
 ('ReCiPe Midpoint (H)', 'ionising radiation', 'IRP_HE'),
 ('ReCiPe Midpoint (H)', 'marine ecotoxicity', 'METPinf'),
 ('ReCiPe Midpoint (H)', 'marine eutrophication', 'MEP'),
 ('ReCiPe Midpoint (H)', 'metal depletion', 'MDP'),
 ('ReCiPe Midpoint (H)', 'natural land transformation', 'NLTP'),
 ('ReCiPe Midpoint (H)', 'ozone depletion', 'ODPinf'),
 ('ReCiPe Midpoint (H)', 'particulate matter formation', 'PMFP'),
 ('ReCiPe Midpoint (H)', 'photochemical oxidant formation', 'POFP'),
 ('ReCiPe Midpoint (H)', 'terrestrial acidification', 'TAP100'),
 ('ReCiPe Midpoint (H)', 'terrestrial ecotoxicity', 'TETPinf'),
 ('ReCiPe Midpoint (H)', 'urban land 

select the matching categories, we only use the IPCC CC instead of the RECIPE one, however they have the same unit so we can use the same mid to endpoint factor

we use the RECIPE 1.11
    1. we developed a land occupation method ourselves to include it in the Ecosystem toxicity
    2. we did a crude matching of land occupation categories to urban and rural
    3. we took the average of each of the two categories
    4. for land transformation we use to and from forest and take the average terrestrial species per m2

In [19]:
categories = ['ALO', 'CC', 'FD', 'FET', 'FE',
              'HT', 'IR', 'MET', 'ME', 'MD',
              'NLT', 'OD', 'PMF', 'POF', 'TA',
              'TET', 'ULO', 'WD']
categories_total = ['ALO', 'CC', 'FD', 'FET', 'FE',
              'HT', 'IR', 'MET', 'ME', 'MD',
              'NLT', 'OD', 'PMF', 'POF', 'TA',
              'TET', 'ULO', 'WD','total'] 

perform LCA for all elec markets

In [ ]:
electricity_markets_LCIA= {}
se_technologies_LCIA= {}

for db_name in database_dict:
    for category in categories:
        electricity_markets_LCIA[(database_dict[db_name]['year'], database_dict[db_name]['scenario'], category)] = lcahelp.LCA_to_df(electricity_markets[db_name], cats = [category])
        se_technologies_LCIA[(database_dict[db_name]['year'], database_dict[db_name]['scenario'], category)]     = lcahelp.LCA_to_df(se_technologies[db_name], cats = [category])

load the mid to end characterization factors
calculate all results and aggregate them to DALYS and ET

In [21]:
mid_end = pd.read_csv("data\\recipe_mid_to_end.csv", sep = ';', index_col = 0) 
mid_end


,Midpoint impact category,Unit,HH_H,ED_H,RA_H
Code,,,,,
CC,Climate change,kg CO2 eq,9.280000e-07,2.800000e-09,0.0000
OD,Ozone depletion,kg CFC-11 eq,5.310000e-04,3.030000e-03,0.0000
TA,Terrestrial acidification,kg SO2 eq,0.000000e+00,2.120000e-07,0.0000
FE,Freshwater eutrophication,kg P eq,0.000000e+00,4.440000e-08,0.0000
HT,Human toxicity,"kg 1,4-DB eq",7.000000e-07,0.000000e+00,0.0000
POF,Photochemical oxidant formation,kg NMVOC,3.900000e-08,0.000000e+00,0.0000
PMF,Particulate matter formation,kg PM10 eq,2.600000e-04,0.000000e+00,0.0000
TET,Terrestrial ecotoxicity,"kg 1,4-DB eq",0.000000e+00,1.510000e-07,0.0000
FET,Freshwater ecotoxicity,"kg 1,4-DB eq",0.000000e+00,8.610000e-10,0.0000


In [22]:
end_categories = ['HH_H', 'ED_H', 'RA_H'] 
    

In [23]:
end_mid_electricity_markets = {}
end_mid_se_technologies = {}
for db_name in database_dict:        
    for end_category in end_categories:
        for category in categories:
                    
            end_mid_electricity_markets[(database_dict[db_name]['year'], database_dict[db_name]['scenario'],  end_category, category)] = electricity_markets_LCIA[(database_dict[db_name]['year'], database_dict[db_name]['scenario'], category)] * mid_end.loc[category, end_category]
            end_mid_se_technologies[(database_dict[db_name]['year'], database_dict[db_name]['scenario'],  end_category, category)]     = se_technologies_LCIA[(database_dict[db_name]['year'], database_dict[db_name]['scenario'], category)] * mid_end.loc[category, end_category]                                                                                 


In [ ]:
end_end_electricity_markets = pd.concat(end_mid_electricity_markets, names=['period','scenario','end_category', 'category', 'variable'])
end_end_se_technologies     = pd.concat(end_mid_se_technologies, names=['period','scenario','end_category', 'category', 'variable'])

sum up all end categories

In [26]:
end_end_electricity_markets = end_end_electricity_markets.groupby(['period','scenario', 'end_category', 'variable']).sum()
end_end_se_technologies     = end_end_se_technologies.groupby(['period','scenario', 'end_category', 'variable']).sum()

In [27]:
end_end_electricity_markets['total'] = end_end_electricity_markets.sum(axis = 1)
end_end_se_technologies['total']     = end_end_se_technologies.sum(axis = 1)

In [28]:
end_end_electricity_markets.to_csv('interm\\lca_electricity_markets_per_tec' + project + '.csv', sep=';')
end_end_se_technologies.to_csv('interm\\lca_se_technologies_per_tec' + project + '.csv', sep=';')

In [29]:
end_end_electricity_markets  = pd.read_csv('interm\\lca_electricity_markets_per_tec' + project + '.csv', sep = ';') 
end_end_se_technologies      = pd.read_csv('interm\\lca_se_technologies_per_tec' + project + '.csv', sep = ';') 

add location

In [30]:
end_end_electricity_markets['ecoinvent'] = end_end_electricity_markets['variable'].str.split(",").str[2].str.split("'").str[1].str.split("-").str[0]
end_end_se_technologies['ecoinvent']     = end_end_se_technologies['variable'].str.split(",").str[-1].str.split("'").str[1].str.split("-").str[0]

match REMIND locations

In [31]:
end_end_electricity_markets['variable'] = end_end_electricity_markets['variable'].str.split(",").str[0].str.split("'").str[1]
end_end_se_technologies['variable'] = end_end_se_technologies['variable'].str.split("'").str[1]

In [32]:
region_mapping_elec     = pd.read_csv("data\\regionmappingREMIND_noFullISO_elec.csv", sep = ';', index_col = 0)
region_mapping_se       = pd.read_csv("data\\regionmappingREMIND_noFullISO_se.csv", sep = ';', index_col = 0)
region_mapping_se_world = pd.read_csv("data\\regionmappingREMIND_noFullISO_se_world.csv", sep = ';', index_col = False)

In [33]:
end_end_electricity_markets = pd.merge(end_end_electricity_markets, region_mapping_elec, on= "ecoinvent", how= "inner")
end_end_se_technologies = pd.merge(end_end_se_technologies, region_mapping_se, on= "ecoinvent", how= "inner")

average over multiple ecoinvent locations, we have India with several grids for example

In [34]:
#end_end_electricity_markets.groupby(['period','scenario', 'end_category', 'variable','ecoinvent', 'CountryCode', 'RegionCode']).mean()
#end_end_se_technologies.groupby(['period','scenario', 'end_category', 'variable','ecoinvent', 'CountryCode', 'RegionCode']).mean()

In [35]:
end_end_electricity_markets = end_end_electricity_markets.groupby(['period','scenario', 'end_category', 'variable', 'RegionCode']).mean()
end_end_se_technologies = end_end_se_technologies.groupby(['period','scenario', 'end_category', 'variable', 'RegionCode']).mean()

## put the world values here for all REMIND regions for the non electricity se technologies

In [36]:
end_end_se_technologies.to_csv('interm\\lca.csv', sep=';')
end_end_se_technologies = pd.read_csv("interm\\lca.csv", sep = ';') 
end_end_electricity_markets.to_csv('interm\\lca.csv', sep=';')
end_end_electricity_markets = pd.read_csv("interm\\lca.csv", sep = ';')

In [37]:
end_end_se_technologies = pd.merge(end_end_se_technologies, region_mapping_se_world,  left_on=['RegionCode'], right_on=['World'], how= "right")

In [38]:
end_end_se_technologies['RegionCode'] = end_end_se_technologies['Remind']
#delte World and -- column
#end_end_se_technologies = end_end_se_technologies.drop('World', 1)
end_end_se_technologies = end_end_se_technologies.drop('World', 1)
end_end_se_technologies = end_end_se_technologies.drop('Remind', 1)

In [124]:
end_end = end_end_electricity_markets.append(end_end_se_technologies)

In [125]:
pd.DataFrame(end_end)

,period,scenario,end_category,variable,RegionCode,ALO,CC,FD,FE,FET,...,MET,NLT,OD,PMF,POF,TA,TET,ULO,WD,total
0,2015,ecoinvent,ED_H,market for electricity,AFR,2.575602e-11,1.231210e-09,0.000000,5.109007e-12,2.191015e-12,...,4.595624e-13,1.820439e-10,1.272104e-10,0.000000e+00,0.000000e+00,5.459689e-10,4.826480e-12,2.898852e-11,1.258658e-11,2.166350e-09
1,2015,ecoinvent,ED_H,market for electricity,CHN,1.736818e-10,2.461103e-09,0.000000,1.218982e-11,5.786859e-12,...,1.128820e-12,8.161865e-11,2.920762e-11,0.000000e+00,0.000000e+00,9.323017e-10,4.077971e-12,1.697356e-10,3.901464e-11,3.909847e-09
2,2015,ecoinvent,ED_H,market for electricity,EUR,8.096477e-10,1.594210e-09,0.000000,2.440525e-11,7.904036e-12,...,1.578816e-12,1.669798e-10,1.817344e-10,0.000000e+00,0.000000e+00,7.463813e-10,4.396165e-12,4.879403e-11,3.467755e-11,3.620709e-09
3,2015,ecoinvent,ED_H,market for electricity,IND,2.393309e-10,3.584665e-09,0.000000,2.804944e-11,9.988024e-12,...,1.937514e-12,2.755793e-10,5.275627e-11,0.000000e+00,0.000000e+00,1.000343e-09,3.939699e-12,2.310763e-10,6.058372e-11,5.488250e-09
4,2015,ecoinvent,ED_H,market for electricity,JPN,3.894061e-10,2.121195e-09,0.000000,6.239945e-12,3.469888e-12,...,5.384341e-13,5.175625e-10,7.366588e-11,0.000000e+00,0.000000e+00,6.535029e-10,6.381841e-12,7.964519e-11,2.494155e-11,3.876549e-09
5,2015,ecoinvent,ED_H,market for electricity,LAM,4.645821e-10,1.301083e-09,0.000000,2.652232e-12,1.480078e-12,...,3.235998e-13,2.752433e-10,1.635933e-10,0.000000e+00,0.000000e+00,5.614828e-10,6.293275e-12,5.182357e-11,1.244284e-11,2.841000e-09
6,2015,ecoinvent,ED_H,market for electricity,MEA,2.483865e-11,1.826507e-09,0.000000,2.801390e-12,1.799817e-12,...,3.566482e-13,3.046267e-10,1.725399e-10,0.000000e+00,0.000000e+00,5.444692e-10,5.825530e-12,3.120872e-11,2.313846e-11,2.938112e-09
7,2015,ecoinvent,ED_H,market for electricity,OAS,8.888417e-11,1.740995e-09,0.000000,1.838587e-11,6.553923e-12,...,1.263095e-12,1.467440e-10,7.909401e-11,0.000000e+00,0.000000e+00,5.462239e-10,3.002173e-12,7.416437e-11,3.041184e-11,2.735722e-09
8,2015,ecoinvent,ED_H,market for electricity,ROW,1.191581e-10,1.378619e-09,0.000000,1.962748e-11,6.470766e-12,...,1.275289e-12,2.018672e-10,1.017851e-10,0.000000e+00,0.000000e+00,6.999639e-10,3.709511e-12,4.043809e-11,3.223118e-11,2.605145e-09
9,2015,ecoinvent,ED_H,market for electricity,RUS,9.690186e-11,1.926802e-09,0.000000,9.595213e-12,3.715968e-12,...,6.714672e-13,2.780886e-10,1.647226e-10,0.000000e+00,0.000000e+00,4.125192e-10,2.354045e-12,3.873108e-11,3.620807e-11,2.970310e-09


join with REMIND mapping and energy content conversion

In [126]:
# join with REMIND mapping
## read REMIND mapping
## join
## divide with convertion to go to EJ

In [127]:
tech_mapping = pd.read_csv("data\\ecoinventtoREMINDtechmap.csv", sep = ';', index_col = False) 

In [128]:
end_end = pd.merge(end_end, tech_mapping, left_on=['variable'], right_on=['ecoinvent'], how= "left")

In [129]:
end_end_out.to_csv('results\\lca_per_tec_prelim_' + project  + '.csv', sep=';')

In [130]:
end_end['total'] = end_end['total'] / end_end['conv']
end_end[categories]  = end_end[categories].divide(end_end["conv"], axis="index")

cleanup

In [ ]:
end_end_out = end_end[['period','scenario','REMIND','end_category', 'RegionCode']]
end_end_out[categories] = end_end[categories]
end_end_out['total'] = end_end['total']

In [132]:
end_end_out = pd.melt(end_end_out, id_vars=['period', 'scenario','REMIND','RegionCode','end_category'], value_vars=categories_total)

write results

In [133]:
end_end_out.to_csv('results\\lca_per_tec_' + project  + '.csv', sep=';')

# read all REMIND mifs and calculate the actual model specific LCA including the monetization
1. read all mifs and append them
2. join with end_end_out and multiply
3. join with vsl_coeff
    1. join vsl_coeff with regions mapping and average over region

read REMIND results

In [134]:
mif_names = os.listdir("data\\mifs\\")

mifs from wide to long

In [135]:
mifs = pd.concat( [ pd.read_csv("data\\mifs\\" + mif, sep = ';') for mif in mif_names ] )

define colnames that are years

In [136]:
mifs = mifs.drop('Unnamed: 24', 1)

In [137]:
year_cols = [col for col in mifs.columns if '2' in col]

In [138]:
mifs = pd.melt(mifs, id_vars=['Model', 'Scenario','Region','Variable', 'Unit'], value_vars=year_cols)

rename columns

In [139]:
mifs.columns=['model','scenario','region','tech','unit','period','SE']
mifs = mifs.drop('model', 1)
end_end_out.columns=['period','scenario','tech','region','end_category','category','lca']

join LCA results with the mifs

In [140]:
mifs['period']=mifs['period'].astype(int)
end_end_out['period']=end_end_out['period'].astype(int)

map scenarios

In [ ]:
end_end_out['scenario'][end_end_out['scenario'] == 'ecoinvent'] = 'SSP2-Ref'
end_end_out['scenario'][end_end_out['scenario'] == 'INDC'] = 'SSP2-INDC'
end_end_out['scenario'][end_end_out['scenario'] == 'RCP26'] = 'SSP2-26'
end_end_out['scenario'][end_end_out['scenario'] == 'Ref'] = 'SSP2-Ref'
end_end_out['scenario'][end_end_out['scenario'] == 'coal_exit'] = 'SSP2-INDC_coal_exit'

In [142]:
end_end_out = pd.merge(end_end_out, mifs, left_on=['period','scenario','tech','region'], right_on=['period','scenario','tech','region'], how= "left")
end_end_out['lca_impact'] = end_end_out['lca'] * end_end_out['SE']

write results

In [143]:
end_end_out.to_csv('results\\lca_REMIND_' + project  + '.csv', sep=';')

## read the VSL and multiply the HH and ED values with the VSL and end category

read the vsl coeff and do the average over REMIND regions
    map to REMIND regions
join vsl with end_end_out 
multiply with monetization for ED and HH and R?

In [144]:
vsl_coeff = pd.read_csv("data//vsl_coeff.csv", sep = ',', index_col = 0) 

join with REMIND region mapping

## condense to REMIND region results

In [145]:
vsl_coeff = pd.merge(vsl_coeff, region_mapping_elec, left_on= "Region", right_on='CountryCode', how= "left")
vsl_coeff = vsl_coeff.groupby(['RegionCode','Year','Data2']).mean()
vsl_coeff.columns = ['vsl_coeff']

## join end_end_out and vsl_coeff

In [146]:
end_end_out = pd.merge(end_end_out, vsl_coeff, left_on= ["period","region"], right_on=["Year","RegionCode"], how= "left")

## add the monetization

#VSL from OECD (2012) Mortality risk valuation in environment, health and transport policies. OECD Publishing
#divded by the 2017 ration of mortality to DALYS, all cause all ages
u_DALY <- 3600000 / 30.4 [$2005]

In [147]:
#HH_H [DALYS] = 118421.1  

#ED_H [species.yr] = 50447778 , Kiuk 2008 gives a value from the NEEDs project of 0.25 €2004/PDF/m2/year for the EU 25 based on restauration cost of from Germany to go from integreated arable to organic arable (the lowest cost to transform land)

#@Ott2007 gives a spatially weighted average to transform "Built Up Land" into
#other land for EU-25: 1.625 $2005/PDF/m2

#this equals 0.165 $2005/PDF/m2/year if we apply 10a discounting time

#terrestrial species density: 1.48 E-8 [1/m2][3] global data
#	CF PDF = SD PDF, only terrestrial
#	CF [↕species.yr] = PDF.m2.yr * species/m²
#	€/ CF [species.yr]= CF [↕species.yr] *1/( species/m²)
#1/1.48 E-8 [1/m2] = 67567568 [1/(species/m²)]
#0.165$ [2005] per m² per year * 67567568 [1/(species/m²)] = 111486487.2 $/species year


#RD_H [$] = 1.09

In [ ]:
end_end_out['HH_H_marginal_cost'] = 118421.1
end_end_out['ED_H_marginal_cost'] = 111486487.2
end_end_out['RA_H_marginal_cost'] = 1.09
end_end_out['HH_H_cost_total']=0
end_end_out['ED_H_cost_total']=0
end_end_out['RA_H_cost_total']=0
end_end_out['HH_H_cost_total'][end_end_out['end_category'] == 'HH_H'] = end_end_out['HH_H_marginal_cost'] * end_end_out['lca_impact']
end_end_out['ED_H_cost_total'][end_end_out['end_category'] == 'ED_H'] = end_end_out['ED_H_marginal_cost'] * end_end_out['lca_impact']
end_end_out['RA_H_cost_total'][end_end_out['end_category'] == 'RA_H'] = end_end_out['RA_H_marginal_cost'] * end_end_out['lca_impact']

end_end_out['HH_H_cost_total_vsl']=0
end_end_out['ED_H_cost_total_vsl']=0
end_end_out['RA_H_cost_total_vsl']=0
end_end_out['HH_H_cost_total_vsl'][end_end_out['end_category'] == 'HH_H'] = end_end_out['HH_H_cost_total'] * end_end_out['vsl_coeff']
end_end_out['ED_H_cost_total_vsl'][end_end_out['end_category'] == 'ED_H'] = end_end_out['ED_H_cost_total'] * end_end_out['vsl_coeff']
end_end_out['RA_H_cost_total_vsl'][end_end_out['end_category'] == 'RA_H'] = end_end_out['RA_H_cost_total'] * end_end_out['vsl_coeff']

## write all categories and total

In [149]:
end_end_out.to_csv('results\\lca_REMIND_monetization_' + project  + '.csv', sep=';')

In [123]:
#run the policy_cost.R to get the mitigation cost
# run the calclca.R to get the endpoint scors
# run the calclca_cost.R to add the AP to the LCA and plot some cost graphs, here the mitigation cost are added